In [1]:
import linguamind.nlp as nlp

In [2]:
import linguamind.linalg as la

In [4]:
text = nlp.Text("hello world","BIIIIOBIIII")

In [5]:
vocab = text.getVocab("tokens")

In [1]:
2+2

4

In [7]:
# text = lm.Text("hello world")

In [8]:
text = lm.Text("hello world","BIIIIOBIIII")

In [9]:
vocab = text.getVocab("tokens")

In [12]:
for i in xrange(vocab.size):
    print vocab.getTermAtIndex(i).letters

hello
world


In [11]:
text.getSequence("tokens")

(0, 1)

#NLP Components

Two Types of Component:
- word level
- sentence level
- document level

### Data Model

In [22]:
c = lm.Text("/my/folder/of/files") # atempts to autodetect file type
c = lm.Text(path="myfile.txt",type="raw")
c = lm.Text(path="myfile.txt",type="csv-token") # one token per line in CSV
c = lm.Text(path="myfile.txt",type="csv-segment") # text segment per line in CSV
c = lm.Text(path="myfile.txt",type="prefix__label__") # one sentence per line with special labels
c = lm.Text(path="myfile.txt",type="json")

# document level raw text
d1 = lm.Text("This is my sentence.")
d2 = lm.Text("myfile.txt")

c = lm.Text([d1,d2]) #form documents into Corpus

# document level text with labels
contents = {}
contents["text"] = "This is my sentence. This is another one."
contents["date"] = 1997
d = lm.Text(contents)

# sentence level text with labels
contents = {}
contents["text"] = "This is my sentence."
contents["sentiment"] = "Positive"
contents["speaker"] = "John Locke"
d = lm.Text(contents)

### Applying Analytics

In [17]:
tok = lm.Segmenter(name="tokenizer",lang="en")
eos = lm.Segmenter(name="eos",lang="en")

# document level text with labels
contents = {}
contents["text"] = "This is my sentence. This is another one."
contents["date"] = 1997
d = lm.Text(contents)

tok.segment(d)
eos.segment(d)

print d["text"] # "This is my sentence. This is another one."
print d["tokens"] # ["This", "is"...
print d["sentences"] # [["This", "is", "my", "sentence","."],["This","is","another","one","."]]                
                     
pos = lm.Classifier(name="pos")
pos.predict(d)

print d["sentences"] # [[{"token":"This","POS":"NNP"},...                

sentiment = lm.Classifier(name="sentiment")
sentiment.predict(d)
print d["sentences"] # [{"sentiment":"Positive","tokens":[{"token":"This","POS":"NNP"},...

###Creating Analytics

In [ ]:
# Creating a Segmenter (tokenizer)
contents = {}
contents["text"] = "I like pie."
contents["char_segs"] = "BOBIIIOBIIB"
d = lm.Text(contents)

feats=[["text_-2","text_-1"],"text_-1","text_0","text_1",["text_1","text_2"]]

ambiguity_hash = lm.AmbiguityHash(d,feats,"char_segs",threshold=0.95,min_count=5)

percept_model = lm.ml.Sequential()
percept_model.add(lm.ml.Linear(size=[d.getVocab("text").size() * 10],encoding="hashtable"))
percept_model.add(lm.ml.LogSoftmax())

percept_loss = lm.ml.PerceptronLoss()
percept_optimizer = lm.ml.optim.PerceptronUpdate()
percept_searcher = lm.ml.search.BeamSearch(beam=5)

# where available in leau of calling all the forward/backprop logic from Python slowing things down
tokenizer = lm.ml.trainer.FastSegmentationTrainer(d,"char_segs",feats,percept_model,percept_loss,percept_searcher,inherits=[ambiguity_hash],threads=50)

tokenizer.segment(d,"tokens") #generates new vocab for what was segmented

eos = lm.pretrained.Segmenter(name="eos",lang="en")
eos.segment(d,"sentences",ignore_vocab=True) # does not generate a new vocab for what was segmented

# https://github.com/oxford-cs-ml-2015/practical6/blob/master/train.lua
lstm_hidden = 50
seq_length = 16

layers = list()
for i in range(seq_length):
    layer = {}
    
    inputs = {}
    layer['embed'] = lm.ml.Embedding(d.getVocab("token").size(),lstm_hidden)
    layer['lstm'] = lm.ml.LSTM(lstm_hidden,input_x=layer['embed'],input_h=layers[i-1]['lstm']['h'])
    layer['softmax'] = lm.ml.LogElasticHierarchicalSoftmax(input=layer['lstm']['h'],size=(lstm_hidden,d.getVocab("token").size()),sample_rate=0.001)
    layer['criterion'] = lm.ml.LogElasticHierarchicalSoftmaxLoss()
    layers.append(layer)

word_language_model_searcher = lm.ml.search.BeamSearch(beam=5)
percept_optimizer = lm.ml.optim.SGD()
word_language_model = lm.ml.trainer.FastLanguageModelTrainer(d,layers,word_language_model_searcher,percept_optimizer,predict_on="token",bound_on="sentence",threads=50)

#ML Components

In [18]:
#Ambiguity Hashing

In [19]:
#Perceptron 

In [20]:
#LSTM 

In [ ]:
#Neural Index